In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import fitz
from fitz import Rect

from mozilla_sec_eia.utils.pdf import extract_pdf_data_from_page
from mozilla_sec_eia.utils.pdf import render_page

In [ ]:
filepath = Path("../sec10k_filings/pdfs/40545-2004q1-40545-0000040545-04-000013.pdf")
assert filepath.exists()

In [ ]:
doc = fitz.Document(str(filepath))
doc.is_pdf

Multi page PDFs show up as separate indices in doc.

In [ ]:
len(doc)

In [ ]:
pg = doc[1]
extracted = extract_pdf_data_from_page(pg)
extracted.keys()

In [ ]:
txt = extracted['pdf_text']
img_info = extracted['image']
pg_meta = extracted['page']
txt.shape, img_info.shape, pg_meta.shape

In [ ]:
pg_meta

In [ ]:
pg.rect.width

In [ ]:
pg.rect.height

In [ ]:
pg.rect

Try combining both pages into a separate PDF with one longer page.

In [ ]:
combined_width = 0
combined_height = 0
rects = []
for page in doc:
    pg_width = page.rect.width
    combined_width = max(combined_width, pg_width)
    # instead of using page height directly, use the height of the last word + a buffer
    full_pg_height = page.rect.height
    extracted = extract_pdf_data_from_page(page)
    pg_txt_height = extracted['pdf_text'].bottom_right_y_pdf.max() + (full_pg_height/100)
    # Translate this page down by the height of the previous page
    rects.append(Rect(0, combined_height, pg_width, combined_height + pg_txt_height))
    page.set_cropbox(Rect(0, 0, pg_width, pg_txt_height))
    combined_height += pg_txt_height

output_pdf = fitz.open()
combined_page = output_pdf.new_page(width=combined_width, height=combined_height)

for i in range(len(doc)):
    combined_page.show_pdf_page(rects[i], doc, i)

In [ ]:
extracted = extract_pdf_data_from_page(combined_page)
extracted.keys()

In [ ]:
txt = extracted['pdf_text']
img_info = extracted['image']
pg_meta = extracted['page']
txt.shape, img_info.shape, pg_meta.shape

In [ ]:
txt

In [ ]:
full_pg_img = render_page(combined_page)

In [ ]:
full_pg_img